# Calendarios de Pago — Programas para el Bienestar

Construcción del DataFrame de calendarios de dispersión verificados en **programasparaelbienestar.gob.mx** (comunicado por comunicado).

| columna | tipo | descripción |
|---------|------|-------------|
| `grupo` | str | `'pensiones'` \| `'becas'` \| `'sembrando_vida'` |
| `programa` | str | nombre del programa específico |
| `fecha` | Timestamp | fecha de depósito |
| `letra` | str | inicial del primer apellido, o `'TODOS'` |

## Notas de cobertura y diseño

```
CALENDARIOS DE PAGO - PROGRAMAS PARA EL BIENESTAR
Fuente: programasparaelbienestar.gob.mx (verificado comunicado por comunicado)
Fecha de construcción: 2026-02-25

ESTRUCTURA DEL DATAFRAME:
    grupo    : 'pensiones' | 'becas' | 'sembrando_vida'
    programa : nombre del programa específico
    fecha    : fecha de depósito (pd.Timestamp)
    letra    : letra inicial del primer apellido ('A', 'B', ..., 'TODOS')

NOTAS DE COBERTURA Y DISEÑO:
──────────────────────────────────────────────────────────────────────────────
GRUPO PENSIONES (adultos_mayores, discapacidad, madres_trabajadoras)
  - Cobertura: 2023-03 a 2026-01 (bimestres verificados).
  - Los tres programas comparten EXACTAMENTE el mismo calendario por letra.
  - mujeres_bienestar: primer pago nov-2025 con calendario propio (4 grupos);
    desde 2026-01 se integra al calendario compartido de pensiones.
  - El periodo '2023-03' cubre el bimestre mar-abr 2023 y es el más antiguo
    disponible como tabla en el sitio (pagos previos no tenían tabla pública).

GRUPO BECAS (beca_basica / beca_rita_cetina, beca_media_superior, beca_superior)
  - 2023: NO hay calendario por letra en el sitio oficial. Los pagos se hacían
    por emisiones anuales (1ª, 2ª, 3ª emisión) y cada beneficiario consultaba
    su fecha individual via CURP. No se incluyen en el DataFrame.
  - 2024-12: único pago del año con calendario por letra; los tres niveles
    comparten la misma tabla (nombrado 'beca_basica' en 2024).
  - 2025 en adelante: beca_basica → beca_rita_cetina; beca_media_superior →
    Benito Juárez EMS; beca_superior → Jóvenes Escribiendo el Futuro (JEF).
  - oct 2025: Rita Cetina (6-17 oct), Benito Juárez EMS (20-27 oct),
    JEF = un solo día sin escalonamiento (29 oct → letra='TODOS').
  - 2026-02: solo Rita Cetina publicada a la fecha de construcción.
    Benito Juárez EMS y JEF se esperan; se incluirán cuando se publiquen.

GRUPO SEMBRANDO VIDA
  - 2023: compartía EXACTAMENTE el mismo calendario por letra que pensiones
    (confirmado en comunicados de mar y may 2023).
  - 2024+: pago mensual en fecha única ('TODOS'), sin escalonamiento por letra.
    Solo se han podido verificar las fechas anunciadas explícitamente en el
    sitio. Las fechas mensuales no anunciadas no se incluyen para no inferir.
    Fechas verificadas: sep-2024 (9 sep), nov-2024 (12 nov), nov-2025 (7 nov).
──────────────────────────────────────────────────────────────────────────────
```

## 0. Imports

In [1]:
import pandas as pd


## 1. Datos fuente — Pensiones

Calendarios bimestrales verificados desde **mar-abr 2023** hasta **ene-feb 2026**. Cada entrada `(fecha_str, [letras])` mapea una fecha de depósito a las letras que cobran ese día. Los tres programas principales (adultos mayores, discapacidad, madres trabajadoras) comparten el mismo calendario en todos los periodos. `mujeres_bienestar` tiene su propio calendario en nov-2025 y se integra al calendario compartido desde ene-feb 2026.

In [ ]:
# =============================================================================
# DATOS FUENTE: PENSIONES
# Cada entrada: (fecha_str, [letras])
# =============================================================================

_PENSIONES_RAW = {
    # ── mar-abr 2023 ──────────────────────────────────────────────────────────
    '2023-03': [
        ('2023-03-02', ['A', 'B']),
        ('2023-03-03', ['C']),
        ('2023-03-06', ['D', 'E', 'F']),
        ('2023-03-07', ['G']),
        ('2023-03-08', ['H', 'I', 'J', 'K', 'L']),
        ('2023-03-09', ['M']),
        ('2023-03-10', ['N', 'Ñ', 'O', 'P', 'Q']),
        ('2023-03-13', ['R']),
        ('2023-03-14', ['S', 'T', 'U']),
        ('2023-03-15', ['V', 'W', 'X', 'Y', 'Z']),
    ],
    # ── may-jun 2023 ──────────────────────────────────────────────────────────
    '2023-05': [
        ('2023-05-03', ['A', 'B']),
        ('2023-05-04', ['C']),
        ('2023-05-05', ['D', 'E', 'F']),
        ('2023-05-09', ['G']),
        ('2023-05-10', ['H', 'I', 'J', 'K', 'L']),
        ('2023-05-11', ['M']),
        ('2023-05-12', ['N', 'Ñ', 'O', 'P', 'Q']),
        ('2023-05-15', ['R']),
        ('2023-05-16', ['S', 'T', 'U']),
        ('2023-05-17', ['V', 'W', 'X', 'Y', 'Z']),
    ],
    # ── jul-ago 2023 ──────────────────────────────────────────────────────────
    '2023-07': [
        ('2023-07-04', ['A', 'B']),
        ('2023-07-05', ['C']),
        ('2023-07-06', ['D', 'E', 'F']),
        ('2023-07-07', ['G']),
        ('2023-07-10', ['H', 'I', 'J', 'K', 'L']),
        ('2023-07-11', ['M']),
        ('2023-07-12', ['N', 'Ñ', 'O', 'P', 'Q']),
        ('2023-07-13', ['R']),
        ('2023-07-14', ['S', 'T', 'U']),
        ('2023-07-17', ['V', 'W', 'X', 'Y', 'Z']),
    ],
    # ── sep-oct 2023 ──────────────────────────────────────────────────────────
    '2023-09': [
        ('2023-09-07', ['A']),
        ('2023-09-08', ['B']),
        ('2023-09-11', ['C']),
        ('2023-09-12', ['D', 'E', 'F']),
        ('2023-09-13', ['G']),
        ('2023-09-14', ['H']),
        ('2023-09-18', ['I', 'J', 'K']),
        ('2023-09-19', ['L']),
        ('2023-09-20', ['M']),
        ('2023-09-21', ['N', 'Ñ', 'O']),
        ('2023-09-22', ['P', 'Q']),
        ('2023-09-25', ['R']),
        ('2023-09-26', ['S']),
        ('2023-09-27', ['T', 'U']),
        ('2023-09-28', ['V', 'W', 'X', 'Y', 'Z']),
    ],
    # ── nov-dic 2023 ──────────────────────────────────────────────────────────
    '2023-11': [
        ('2023-11-06', ['A']),
        ('2023-11-07', ['B']),
        ('2023-11-08', ['C']),
        ('2023-11-09', ['C']),
        ('2023-11-10', ['D', 'E', 'F']),
        ('2023-11-13', ['G']),
        ('2023-11-14', ['G']),
        ('2023-11-15', ['H', 'I', 'J', 'K']),
        ('2023-11-16', ['L']),
        ('2023-11-17', ['M']),
        ('2023-11-21', ['M']),
        ('2023-11-22', ['N', 'Ñ', 'O']),
        ('2023-11-23', ['P', 'Q']),
        ('2023-11-24', ['R']),
        ('2023-11-27', ['R']),
        ('2023-11-28', ['S']),
        ('2023-11-29', ['T', 'U']),
        ('2023-11-30', ['V', 'W', 'X', 'Y', 'Z']),
    ],
    # ── ene-feb 2024 ──────────────────────────────────────────────────────────
    '2024-01': [
        ('2024-01-03', ['A']),
        ('2024-01-04', ['B']),
        ('2024-01-05', ['C']),
        ('2024-01-08', ['C']),
        ('2024-01-09', ['D', 'E', 'F']),
        ('2024-01-10', ['G']),
        ('2024-01-11', ['G']),
        ('2024-01-12', ['H', 'I', 'J', 'K']),
        ('2024-01-15', ['L']),
        ('2024-01-16', ['M']),
        ('2024-01-17', ['M']),
        ('2024-01-18', ['N', 'Ñ', 'O']),
        ('2024-01-19', ['P', 'Q']),
        ('2024-01-22', ['R']),
        ('2024-01-23', ['R']),
        ('2024-01-24', ['S']),
        ('2024-01-25', ['T', 'U']),
        ('2024-01-26', ['V', 'W', 'X', 'Y', 'Z']),
    ],
    # ── mar-abr + may-jun 2024 (pago doble, veda electoral) ───────────────────
    '2024-01-electoral': [
        ('2024-01-29', ['A']),
        ('2024-01-30', ['B']),
        ('2024-01-31', ['C']),
        ('2024-02-01', ['C']),
        ('2024-02-02', ['D', 'E', 'F']),
        ('2024-02-06', ['G']),
        ('2024-02-07', ['G']),
        ('2024-02-08', ['H']),
        ('2024-02-09', ['I', 'J', 'K']),
        ('2024-02-12', ['L']),
        ('2024-02-13', ['M']),
        ('2024-02-14', ['M']),
        ('2024-02-15', ['N', 'Ñ', 'O']),
        ('2024-02-16', ['P', 'Q']),
        ('2024-02-19', ['R']),
        ('2024-02-20', ['R']),
        ('2024-02-21', ['S']),
        ('2024-02-22', ['T', 'U']),
        ('2024-02-23', ['V', 'W', 'X', 'Y', 'Z']),
    ],
    # ── jul-ago 2024 ──────────────────────────────────────────────────────────
    '2024-07': [
        ('2024-07-01', ['A']),
        ('2024-07-02', ['B']),
        ('2024-07-03', ['C']),
        ('2024-07-04', ['C']),
        ('2024-07-05', ['D', 'E', 'F']),
        ('2024-07-06', ['G']),
        ('2024-07-08', ['G']),
        ('2024-07-09', ['H', 'I', 'J', 'K']),
        ('2024-07-10', ['L']),
        ('2024-07-11', ['M']),
        ('2024-07-12', ['M']),
        ('2024-07-13', ['N', 'Ñ', 'O', 'P', 'Q']),
        ('2024-07-15', ['R']),
        ('2024-07-16', ['R']),
        ('2024-07-17', ['S']),
        ('2024-07-18', ['T']),
        ('2024-07-19', ['U', 'V', 'W', 'X', 'Y', 'Z']),
    ],
    # ── sep-oct 2024 ──────────────────────────────────────────────────────────
    '2024-09': [
        ('2024-09-02', ['A']),
        ('2024-09-03', ['B']),
        ('2024-09-04', ['C']),
        ('2024-09-05', ['C']),
        ('2024-09-06', ['D', 'E', 'F']),
        ('2024-09-07', ['G']),
        ('2024-09-09', ['G']),
        ('2024-09-10', ['H', 'I', 'J', 'K']),
        ('2024-09-11', ['L']),
        ('2024-09-12', ['M']),
        ('2024-09-13', ['M']),
        ('2024-09-14', ['N', 'Ñ', 'O', 'P', 'Q']),
        ('2024-09-18', ['R']),
        ('2024-09-19', ['S']),
        ('2024-09-20', ['T', 'U', 'V']),
        ('2024-09-21', ['W', 'X', 'Y', 'Z']), 
    ],
    # ── nov-dic 2024 ──────────────────────────────────────────────────────────
    '2024-11': [
        ('2024-11-04', ['A']),
        ('2024-11-05', ['B']),
        ('2024-11-06', ['C']),
        ('2024-11-07', ['C']),
        ('2024-11-08', ['D', 'E', 'F']),
        ('2024-11-11', ['G']),
        ('2024-11-12', ['G']),
        ('2024-11-13', ['H', 'I', 'J', 'K']),
        ('2024-11-14', ['L']),
        ('2024-11-15', ['M']),
        ('2024-11-19', ['M']),
        ('2024-11-20', ['N', 'Ñ', 'O']),
        ('2024-11-21', ['P', 'Q']),
        ('2024-11-22', ['R']),
        ('2024-11-25', ['R']),
        ('2024-11-26', ['S']),
        ('2024-11-27', ['T', 'U', 'V']),
        ('2024-11-28', ['W', 'X', 'Y', 'Z']),
    ],
    # ── ene-feb 2025 ──────────────────────────────────────────────────────────
    '2025-01': [
        ('2025-01-02', ['A']),
        ('2025-01-03', ['B']),
        ('2025-01-04', ['C']),
        ('2025-01-06', ['C']),
        ('2025-01-07', ['D', 'E', 'F']),
        ('2025-01-08', ['G']),
        ('2025-01-09', ['G']),
        ('2025-01-10', ['H', 'I', 'J', 'K']),
        ('2025-01-11', ['L']),
        ('2025-01-13', ['M']),
        ('2025-01-14', ['M']),
        ('2025-01-15', ['N', 'Ñ', 'O']),
        ('2025-01-16', ['P', 'Q']),
        ('2025-01-17', ['R']),
        ('2025-01-18', ['R']),
        ('2025-01-20', ['S']),
        ('2025-01-21', ['T', 'U', 'V']),
        ('2025-01-22', ['W', 'X', 'Y', 'Z']),
    ],
    # ── mar-abr 2025 ──────────────────────────────────────────────────────────
    '2025-03': [
        ('2025-03-03', ['A']),
        ('2025-03-04', ['B']),
        ('2025-03-05', ['C']),
        ('2025-03-06', ['C']),
        ('2025-03-07', ['D', 'E', 'F']),
        ('2025-03-10', ['G']),
        ('2025-03-11', ['G']),
        ('2025-03-12', ['H', 'I', 'J', 'K']),
        ('2025-03-13', ['L']),
        ('2025-03-14', ['M']),
        ('2025-03-17', ['M']),
        ('2025-03-18', ['N', 'Ñ', 'O']),
        ('2025-03-19', ['P', 'Q']),
        ('2025-03-20', ['R']),
        ('2025-03-24', ['R']),
        ('2025-03-25', ['S']),
        ('2025-03-26', ['T', 'U', 'V']),
        ('2025-03-27', ['W', 'X', 'Y', 'Z']),
    ],
    # ── may-jun 2025 ──────────────────────────────────────────────────────────
    '2025-05': [
        ('2025-05-07', ['A']),
        ('2025-05-08', ['B']),
        ('2025-05-09', ['C']),
        ('2025-05-12', ['C']),
        ('2025-05-13', ['D', 'E', 'F']),
        ('2025-05-14', ['G']),
        ('2025-05-15', ['G']),
        ('2025-05-16', ['H', 'I', 'J', 'K']),
        ('2025-05-19', ['L']),
        ('2025-05-20', ['M']),
        ('2025-05-21', ['M']),
        ('2025-05-22', ['N', 'Ñ', 'O']),
        ('2025-05-23', ['P', 'Q']),
        ('2025-05-26', ['R']),
        ('2025-05-27', ['R']),
        ('2025-05-28', ['S']),
        ('2025-05-29', ['T', 'U', 'V']),
        ('2025-05-30', ['W', 'X', 'Y', 'Z']),
    ],
    # ── jul-ago 2025 ──────────────────────────────────────────────────────────
    '2025-07': [
        ('2025-07-01', ['A']),
        ('2025-07-02', ['B']),
        ('2025-07-03', ['C']),
        ('2025-07-04', ['C']),
        ('2025-07-07', ['D', 'E', 'F']),
        ('2025-07-08', ['G']),
        ('2025-07-09', ['G']),
        ('2025-07-10', ['H', 'I', 'J', 'K']),
        ('2025-07-11', ['L']),
        ('2025-07-14', ['M']),
        ('2025-07-15', ['M']),
        ('2025-07-16', ['N', 'Ñ', 'O']),
        ('2025-07-17', ['P', 'Q']),
        ('2025-07-18', ['R']),
        ('2025-07-21', ['R']),
        ('2025-07-22', ['S']),
        ('2025-07-23', ['T', 'U', 'V']),
        ('2025-07-24', ['W', 'X', 'Y', 'Z']),
    ],
    # ── sep-oct 2025 ──────────────────────────────────────────────────────────
    '2025-09': [
        ('2025-09-01', ['A']),
        ('2025-09-02', ['B']),
        ('2025-09-03', ['C']),
        ('2025-09-04', ['C']),
        ('2025-09-05', ['D', 'E', 'F']),
        ('2025-09-08', ['G']),
        ('2025-09-09', ['G']),
        ('2025-09-10', ['H', 'I', 'J', 'K']),
        ('2025-09-11', ['L']),
        ('2025-09-12', ['M']),
        ('2025-09-15', ['M']),
        ('2025-09-17', ['N', 'Ñ', 'O']),
        ('2025-09-18', ['P', 'Q']),
        ('2025-09-19', ['R']),
        ('2025-09-22', ['R']),
        ('2025-09-23', ['S']),
        ('2025-09-24', ['T', 'U', 'V']),
        ('2025-09-25', ['W', 'X', 'Y', 'Z']),
    ],
    # ── nov-dic 2025 ──────────────────────────────────────────────────────────
    '2025-11': [
        ('2025-11-03', ['A']),
        ('2025-11-04', ['B']),
        ('2025-11-05', ['C']),
        ('2025-11-06', ['C']),
        ('2025-11-07', ['D', 'E', 'F']),
        ('2025-11-10', ['G']),
        ('2025-11-11', ['G']),
        ('2025-11-12', ['H', 'I', 'J', 'K']),
        ('2025-11-13', ['L']),
        ('2025-11-14', ['M']),
        ('2025-11-18', ['M']),
        ('2025-11-19', ['N', 'Ñ', 'O']),
        ('2025-11-20', ['P', 'Q']),
        ('2025-11-21', ['R']),
        ('2025-11-24', ['R']),
        ('2025-11-25', ['S']),
        ('2025-11-26', ['T', 'U', 'V']),
        ('2025-11-27', ['W', 'X', 'Y', 'Z']),
    ],
    # ── ene-feb 2026 ──────────────────────────────────────────────────────────
    '2026-01': [
        ('2026-01-05', ['A']),
        ('2026-01-06', ['B']),
        ('2026-01-07', ['C']),
        ('2026-01-08', ['C']),
        ('2026-01-09', ['D', 'E', 'F']),
        ('2026-01-12', ['G']),
        ('2026-01-13', ['G']),
        ('2026-01-14', ['H', 'I', 'J', 'K']),
        ('2026-01-15', ['L']),
        ('2026-01-16', ['M']),
        ('2026-01-19', ['M']),
        ('2026-01-20', ['N', 'Ñ', 'O']),
        ('2026-01-21', ['P', 'Q']),
        ('2026-01-22', ['R']),
        ('2026-01-23', ['R']),
        ('2026-01-26', ['S']),
        ('2026-01-27', ['T', 'U', 'V']),
        ('2026-01-28', ['W', 'X', 'Y', 'Z']),
    ],
}

# Programas que usan el calendario de pensiones en cada periodo
_PENSIONES_PROGRAMAS = {
    '2023-03':            ['adultos_mayores', 'discapacidad', 'madres_trabajadoras', 'sembrando_vida'],
    '2023-05':            ['adultos_mayores', 'discapacidad', 'madres_trabajadoras', 'sembrando_vida'],
    '2023-07':            ['adultos_mayores', 'discapacidad', 'madres_trabajadoras'],
    '2023-09':            ['adultos_mayores', 'discapacidad', 'madres_trabajadoras'],
    '2023-11':            ['adultos_mayores', 'discapacidad', 'madres_trabajadoras'],
    '2024-01':            ['adultos_mayores', 'discapacidad', 'madres_trabajadoras'],
    '2024-01-electoral':  ['adultos_mayores', 'discapacidad', 'madres_trabajadoras'],
    '2024-07':            ['adultos_mayores', 'discapacidad', 'madres_trabajadoras'],
    '2024-09':            ['adultos_mayores', 'discapacidad', 'madres_trabajadoras'],
    '2024-11':            ['adultos_mayores', 'discapacidad', 'madres_trabajadoras'],
    '2025-01':            ['adultos_mayores', 'discapacidad', 'madres_trabajadoras'],
    '2025-03':            ['adultos_mayores', 'discapacidad', 'madres_trabajadoras'],
    '2025-05':            ['adultos_mayores', 'discapacidad', 'madres_trabajadoras'],
    '2025-07':            ['adultos_mayores', 'discapacidad', 'madres_trabajadoras'],
    '2025-09':            ['adultos_mayores', 'discapacidad', 'madres_trabajadoras'],
    '2025-11':            ['adultos_mayores', 'discapacidad', 'madres_trabajadoras'],
    '2026-01':            ['adultos_mayores', 'discapacidad', 'madres_trabajadoras', 'mujeres_bienestar'],
}

# Mujeres Bienestar nov-2025: calendario propio (4 grupos por letra, no por letra individual)
_MUJERES_BIENESTAR_NOV2025 = [
    ('2025-11-18', ['A', 'B', 'C', 'D']),
    ('2025-11-19', ['E', 'F', 'G', 'H', 'I', 'J', 'K']),
    ('2025-11-20', ['L', 'M', 'N', 'Ñ', 'O', 'P', 'Q']),
    ('2025-11-21', ['R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']),
]


## 2. Datos fuente — Becas

Calendarios de Becas Benito Juárez (básica → Rita Cetina), EMS y Superior.  
**2023:** excluido — no existe tabla pública por letra; cada beneficiario consultaba su fecha individual vía CURP.  
**Oct 2025:** los tres programas tienen calendarios distintos (Rita Cetina 6-17 oct, BJ EMS 20-27 oct, JEF 29 oct) almacenados en claves separadas.

In [3]:
_BECAS_RAW = {
    # ── dic 2024: Benito Juárez básica + EMS + superior (último pago del año) ─
    '2024-12': {
        'programas': ['beca_basica', 'beca_media_superior', 'beca_superior'],
        'dias': [
            ('2024-12-02', ['A']),
            ('2024-12-03', ['B']),
            ('2024-12-04', ['C']),
            ('2024-12-05', ['D', 'E', 'F']),
            ('2024-12-06', ['G']),
            ('2024-12-09', ['G']),
            ('2024-12-10', ['H', 'I', 'J', 'K']),
            ('2024-12-11', ['L']),
            ('2024-12-13', ['M']),
            ('2024-12-16', ['M']),
            ('2024-12-17', ['N', 'Ñ', 'O']),
            ('2024-12-18', ['P', 'Q']),
            ('2024-12-19', ['R']),
            ('2024-12-20', ['R']),
            ('2024-12-23', ['S']),
            ('2024-12-24', ['T', 'U', 'V', 'W', 'X', 'Y', 'Z']),
        ],
    },
    # ── feb 2025: Rita Cetina + Benito Juárez EMS + JEF (mismo calendario) ───
    '2025-02': {
        'programas': ['beca_rita_cetina', 'beca_media_superior', 'beca_superior'],
        'dias': [
            ('2025-02-04', ['A']),
            ('2025-02-05', ['B']),
            ('2025-02-06', ['C']),
            ('2025-02-07', ['C']),
            ('2025-02-08', ['D', 'E', 'F']),
            ('2025-02-10', ['G']),
            ('2025-02-11', ['G']),
            ('2025-02-12', ['H', 'I', 'J', 'K']),
            ('2025-02-13', ['L']),
            ('2025-02-14', ['M']),
            ('2025-02-15', ['M']),
            ('2025-02-17', ['N', 'Ñ', 'O']),
            ('2025-02-18', ['P', 'Q']),
            ('2025-02-19', ['R']),
            ('2025-02-20', ['R']),
            ('2025-02-21', ['S']),
            ('2025-02-22', ['T', 'U', 'V']),
            ('2025-02-24', ['W', 'X', 'Y', 'Z']),
        ],
    },
    # ── abr 2025: Rita Cetina + Benito Juárez EMS + JEF (mismo calendario) ───
    '2025-04': {
        'programas': ['beca_rita_cetina', 'beca_media_superior', 'beca_superior'],
        'dias': [
            ('2025-04-01', ['A']),
            ('2025-04-02', ['B']),
            ('2025-04-03', ['C']),
            ('2025-04-04', ['C']),
            ('2025-04-07', ['D', 'E', 'F']),
            ('2025-04-08', ['G']),
            ('2025-04-09', ['G']),
            ('2025-04-10', ['H', 'I', 'J', 'K']),
            ('2025-04-11', ['L']),
            ('2025-04-14', ['M']),
            ('2025-04-15', ['M']),
            ('2025-04-16', ['N', 'Ñ', 'O']),
            ('2025-04-21', ['P', 'Q']),
            ('2025-04-22', ['R']),
            ('2025-04-23', ['R']),
            ('2025-04-24', ['S']),
            ('2025-04-25', ['T', 'U', 'V']),
            ('2025-04-28', ['W', 'X', 'Y', 'Z']),
        ],
    },
    # ── jun 2025: Rita Cetina + Benito Juárez EMS + JEF (mismo calendario) ───
    '2025-06': {
        'programas': ['beca_rita_cetina', 'beca_media_superior', 'beca_superior'],
        'dias': [
            ('2025-06-04', ['A']),
            ('2025-06-05', ['B']),
            ('2025-06-06', ['C']),
            ('2025-06-09', ['C']),
            ('2025-06-10', ['D', 'E', 'F']),
            ('2025-06-11', ['G']),
            ('2025-06-12', ['G']),
            ('2025-06-13', ['H', 'I', 'J', 'K']),
            ('2025-06-16', ['L']),
            ('2025-06-17', ['M']),
            ('2025-06-18', ['M']),
            ('2025-06-19', ['N', 'Ñ', 'O']),
            ('2025-06-20', ['P', 'Q']),
            ('2025-06-23', ['R']),
            ('2025-06-24', ['R']),
            ('2025-06-25', ['S']),
            ('2025-06-26', ['T', 'U', 'V']),
            ('2025-06-27', ['W', 'X', 'Y', 'Z']),
        ],
    },
    # ── oct 2025 (A): Rita Cetina — calendario propio (6-17 oct) ─────────────
    '2025-10-rita': {
        'programas': ['beca_rita_cetina'],
        'dias': [
            ('2025-10-06', ['A', 'B']),
            ('2025-10-07', ['C']),
            ('2025-10-08', ['D', 'E', 'F']),
            ('2025-10-09', ['G']),
            ('2025-10-10', ['H', 'I', 'J', 'K', 'L']),
            ('2025-10-13', ['M']),
            ('2025-10-14', ['N', 'Ñ', 'O', 'P', 'Q']),
            ('2025-10-15', ['R']),
            ('2025-10-16', ['S']),
            ('2025-10-17', ['T', 'U', 'V', 'W', 'X', 'Y', 'Z']),
        ],
    },
    # ── oct 2025 (B): Benito Juárez EMS — calendario propio (20-27 oct) ──────
    '2025-10-bj': {
        'programas': ['beca_media_superior'],
        'dias': [
            ('2025-10-20', ['A', 'B', 'C']),
            ('2025-10-21', ['D', 'E', 'F', 'G']),
            ('2025-10-22', ['H', 'I', 'J', 'K', 'L']),
            ('2025-10-23', ['M', 'N', 'Ñ', 'O']),
            ('2025-10-24', ['P', 'Q', 'R']),
            ('2025-10-27', ['S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']),
        ],
    },
    # ── oct 2025 (C): JEF — un solo día para todos (29 oct) ──────────────────
    '2025-10-jef': {
        'programas': ['beca_superior'],
        'dias': [
            ('2025-10-29', ['TODOS']),
        ],
    },
    # ── dic 2025: Rita Cetina + Benito Juárez EMS + JEF (mismo calendario) ───
    '2025-12': {
        'programas': ['beca_rita_cetina', 'beca_media_superior', 'beca_superior'],
        'dias': [
            ('2025-12-04', ['A', 'B']),
            ('2025-12-05', ['C']),
            ('2025-12-08', ['D', 'E', 'F']),
            ('2025-12-09', ['G']),
            ('2025-12-10', ['H', 'I', 'J', 'K', 'L']),
            ('2025-12-11', ['M']),
            ('2025-12-15', ['N', 'Ñ', 'O', 'P', 'Q']),
            ('2025-12-16', ['R']),
            ('2025-12-17', ['S']),
            ('2025-12-18', ['T', 'U', 'V', 'W', 'X', 'Y', 'Z']),
        ],
    },
    # ── feb 2026: solo Rita Cetina (BJ EMS y JEF sin publicar a la fecha) ─────
    '2026-02-rita': {
        'programas': ['beca_rita_cetina'],
        'dias': [
            ('2026-02-16', ['A', 'B']),
            ('2026-02-17', ['C']),
            ('2026-02-18', ['D', 'E', 'F']),
            ('2026-02-19', ['G']),
            ('2026-02-20', ['H', 'I', 'J', 'K', 'L']),
            ('2026-02-23', ['M']),
            ('2026-02-24', ['N', 'Ñ', 'O', 'P', 'Q']),
            ('2026-02-25', ['R']),
            ('2026-02-26', ['S']),
            ('2026-02-27', ['T', 'U', 'V', 'W', 'X', 'Y', 'Z']),
        ],
    },
}


## 3. Datos fuente — Sembrando Vida (2024+)

A partir de 2024 los pagos se realizan en una sola fecha mensual sin escalonamiento por letra. Solo se incluyen fechas explícitamente anunciadas en comunicados oficiales; no se infieren fechas no publicadas.

In [4]:
_SEMBRANDO_VIDA_FECHAS_UNICAS = [
    '2024-09-09',  # sep 2024 — verificado en comunicado oficial
    '2024-11-12',  # nov 2024 — verificado en comunicado oficial
    '2025-11-07',  # nov 2025 — verificado en comunicado oficial
]


## 4. Función `construir_calendarios()`

Itera sobre los datos fuente y genera un registro por cada combinación `(grupo, programa, fecha, letra)`. Las becas se colapsan a un único `programa='becas'` vía `drop_duplicates`:  
- En periodos con calendario compartido elimina filas redundantes entre programas.  
- En oct-2025 (fechas distintas por programa) conserva todas las filas.

In [5]:
def construir_calendarios() -> pd.DataFrame:
    registros = []

    # ── PENSIONES (incluye sembrando_vida en 2023 y mujeres_bienestar en 2026) ─
    for periodo, dias in _PENSIONES_RAW.items():
        programas = _PENSIONES_PROGRAMAS[periodo]
        for fecha_str, letras in dias:
            fecha = pd.Timestamp(fecha_str)
            for letra in letras:
                for programa in programas:
                    grupo = 'sembrando_vida' if programa == 'sembrando_vida' else 'pensiones'
                    registros.append({
                        'grupo': grupo,
                        'programa': programa,
                        'fecha': fecha,
                        'letra': letra,
                    })

    # ── MUJERES BIENESTAR nov-2025 (calendario propio ese mes) ────────────────
    for fecha_str, letras in _MUJERES_BIENESTAR_NOV2025:
        fecha = pd.Timestamp(fecha_str)
        for letra in letras:
            registros.append({
                'grupo': 'pensiones',
                'programa': 'mujeres_bienestar',
                'fecha': fecha,
                'letra': letra,
            })

    # ── BECAS ─────────────────────────────────────────────────────────────────
    for _, info in _BECAS_RAW.items():
        for fecha_str, letras in info['dias']:
            fecha = pd.Timestamp(fecha_str)
            for letra in letras:
                for programa in info['programas']:
                    registros.append({
                        'grupo': 'becas',
                        'programa': programa,
                        'fecha': fecha,
                        'letra': letra,
                    })

    # ── SEMBRANDO VIDA 2024+ (fecha única sin escalonamiento por letra) ────────
    for fecha_str in _SEMBRANDO_VIDA_FECHAS_UNICAS:
        registros.append({
            'grupo': 'sembrando_vida',
            'programa': 'sembrando_vida',
            'fecha': pd.Timestamp(fecha_str),
            'letra': 'TODOS',
        })

    df = pd.DataFrame(registros)

    # Colapsar los tres programas de becas en uno solo ('becas').
    # En periodos con calendario compartido el drop_duplicates elimina las filas
    # redundantes. En oct-2025, donde los programas tienen fechas distintas,
    # todas las filas se conservan porque no hay duplicados.
    becas = (
        df[df['grupo'] == 'becas']
        .assign(programa='becas')
        [['grupo', 'programa', 'fecha', 'letra']]
        .drop_duplicates()
    )
    df = (
        pd.concat([df[df['grupo'] != 'becas'], becas], ignore_index=True)
        .sort_values(['grupo', 'programa', 'fecha', 'letra'])
        .reset_index(drop=True)
    )
    return df


## 5. Construcción y resumen

Ejecuta la función y muestra estadísticas de cobertura por programa.

In [6]:
df = construir_calendarios()

print("=" * 65)
print("CALENDARIOS BIENESTAR — RESUMEN")
print("=" * 65)
print(f"Total de filas: {len(df):,}")
print(f"Rango de fechas: {df['fecha'].min().date()} → {df['fecha'].max().date()}")
print()

resumen = (
    df.groupby(['grupo', 'programa'])
    .agg(
        fecha_min=('fecha', 'min'),
        fecha_max=('fecha', 'max'),
        n_fechas_unicas=('fecha', 'nunique'),
        n_filas=('fecha', 'count'),
    )
    .reset_index()
)
resumen['fecha_min'] = resumen['fecha_min'].dt.date
resumen['fecha_max'] = resumen['fecha_max'].dt.date
print(resumen.to_string(index=False))
print()
print("Primeras 5 filas del DataFrame:")
print(df.head(5).to_string(index=False))

# Guardar CSV
df.to_csv('calendarios_bienestar.csv', index=False, encoding='utf-8')
print("\nArchivo guardado: calendarios_bienestar.csv")


CALENDARIOS BIENESTAR — RESUMEN
Total de filas: 1,877
Rango de fechas: 2023-03-02 → 2026-02-27

         grupo            programa  fecha_min  fecha_max  n_fechas_unicas  n_filas
         becas               becas 2024-12-02 2026-02-27              107      232
     pensiones     adultos_mayores 2023-03-02 2026-01-28              277      510
     pensiones        discapacidad 2023-03-02 2026-01-28              277      510
     pensiones madres_trabajadoras 2023-03-02 2026-01-28              277      510
     pensiones   mujeres_bienestar 2025-11-18 2026-01-28               22       58
sembrando_vida      sembrando_vida 2023-03-02 2025-11-07               23       57

Primeras 5 filas del DataFrame:
grupo programa      fecha letra
becas    becas 2024-12-02     A
becas    becas 2024-12-03     B
becas    becas 2024-12-04     C
becas    becas 2024-12-05     D
becas    becas 2024-12-05     E

Archivo guardado: calendarios_bienestar.csv


## 6. Consultas de ejemplo

In [7]:
# Fechas de depósito para la letra 'M' en pensiones
df[(df['grupo'] == 'pensiones') & (df['letra'] == 'M')][['programa', 'fecha']].drop_duplicates().sort_values('fecha')


,programa,fecha
244,adultos_mayores,2023-03-09
754,discapacidad,2023-03-09
1264,madres_trabajadoras,2023-03-09
1291,madres_trabajadoras,2023-05-11
271,adultos_mayores,2023-05-11
...,...,...
725,adultos_mayores,2026-01-16
1236,discapacidad,2026-01-19
726,adultos_mayores,2026-01-19
1746,madres_trabajadoras,2026-01-19


In [8]:
# Las tres ventanas de pago de becas en oct-2025
oct25 = df[(df['grupo'] == 'becas') & (df['fecha'].dt.year == 2025) & (df['fecha'].dt.month == 10)]
oct25.sort_values('fecha')


,grupo,programa,fecha,letra
123,becas,becas,2025-10-06,A
124,becas,becas,2025-10-06,B
125,becas,becas,2025-10-07,C
126,becas,becas,2025-10-08,D
127,becas,becas,2025-10-08,E
128,becas,becas,2025-10-08,F
129,becas,becas,2025-10-09,G
130,becas,becas,2025-10-10,H
131,becas,becas,2025-10-10,I
132,becas,becas,2025-10-10,J


In [9]:
# Días de pago de Sembrando Vida disponibles
df[df['grupo'] == 'sembrando_vida'][['fecha', 'letra']].drop_duplicates().sort_values('fecha')


,fecha,letra
1820,2023-03-02,A
1821,2023-03-02,B
1822,2023-03-03,C
1823,2023-03-06,D
1824,2023-03-06,E
1825,2023-03-06,F
1826,2023-03-07,G
1827,2023-03-08,H
1828,2023-03-08,I
1829,2023-03-08,J


In [10]:
df.to_excel('calendarios_bienestar.xlsx', index=False)